# Steady State - Aggregate

**Table of contents**<a id='toc0_'></a>    
- 1. [Setup](#toc1_)    
- 2. [Find steady state](#toc2_)    
  - 2.1. [Compare with data](#toc2_1_)    
- 3. [Speed and error tests](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [330]:
%load_ext autoreload
%autoreload 2

import time
import pickle
import numpy as np
from scipy import optimize

import matplotlib.pyplot as plt   
plt.rcParams.update({"axes.grid":True,"grid.color":"black","grid.alpha":"0.25","grid.linestyle":"--"})
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

from BabyMAKROModel import BabyMAKROModelClass

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. <a id='toc1_'></a>[Setup](#toc0_)

In [331]:
model = BabyMAKROModelClass(name='baseline') # create an instance of the model

## 2. <a id='toc2_'></a>[Find steady state](#toc0_)

In [332]:
par = model.par
ss = model.ss

model.find_ss(do_print=True)    



Labor supply, search and matching:
ss.S/par.N_work = 0.16, ss.L/par.N_work = 0.96, ss.U/par.N_work = 0.04
ss.delta_L = 0.12, ss.m_s = 0.75, ss.m_v = 0.75, ss.v = 6.67
par.sigma_m = 0.42
Capital agency FOC:
ss.r_K = 0.12
Labor agency FOC:
ss.r_ell = 1.01, (ss.LH-ss.ell)/par.N_work*100 = 0.78
Production firm:
ss.P_Y_0 = 0.91, ss.Gamma = 0.54, ss.Y = 68.77, ss.K = 176.13
Capital accumulation:
ss.iota = 17.61, ss.I = 17.61
Government:
ss.B = 0.00, ss.G = 17.19, ss.tau = 0.52
Households:
ss.Aq/par.N = 0.03, ss.real_W = 1.00, ss.C = 24.53, ss.A = 19.73, ss.r_hh = 0.02
Market clearing:
ss.C/ss.Y = 0.36, ss.G/ss.Y = 0.25, ss.I/ss.Y = 0.26, ss.X/ss.Y = 0.60, ss.M/ss.Y = 0.46


### 2.1. <a id='toc2_1_'></a>[Compare with data](#toc0_)

In [333]:
with open('data/ss_aggregate.pickle', 'rb') as f:
    ss_aggregate = pickle.load(f)

for k, v in ss_aggregate.items():
    print(f'{k}')
    print(f'  From data: {v:.2f}')
    print(f'  From model: {getattr(ss, k):.2f}')

M_ratio
  From data: 0.46
  From model: 0.46
X_ratio
  From data: 0.52
  From model: 0.60
C_ratio
  From data: 0.47
  From model: 0.36
G_ratio
  From data: 0.25
  From model: 0.25
I_ratio
  From data: 0.22
  From model: 0.26
K_ratio
  From data: 5.44
  From model: 2.56
L_ratio
  From data: 0.74
  From model: 0.72
U
  From data: 3.93
  From model: 1.67


## 3. <a id='toc3_'></a>[Speed and error tests](#toc0_)

In [334]:
model.set_exo_ss()          
model.set_unknowns_ss()     

**Python evaluation:**

In [335]:
%time model.evaluate_blocks(py=True)

[40.99833507]
[1.00930623]
[68.76664875]
CPU times: total: 1.38 s
Wall time: 1.41 s


**Check errors:**

In [336]:
errors = model.get_errors(do_print=True)

A_R_ini_error       : abs. max = 5.65e-12
Aq_diff             : abs. max = 2.70e-14
FOC_capital_agency  : abs. max = 0.00e+00
FOC_K_ell           : abs. max = 3.55e-15
mkt_clearing        : abs. max = 1.42e-14
PC                  : abs. max = 0.00e+00


In [337]:
if np.allclose(errors,0.0):
    print('errors are zero')
else:
    raise ValueError('errors are NOT zero')

errors are zero


**First numba evaluation:**

In [338]:
%time model.evaluate_blocks()

[82.33028096]
[1.00930623]
[109.15304584]
CPU times: total: 55.4 s
Wall time: 58.7 s


**Time:**

In [339]:
%timeit model.evaluate_blocks()
%timeit model.get_errors()

[123.66222685]
[1.00930623]
[142.99927528]
[164.99417274]
[1.00930623]
[173.2097975]
[206.32611863]
[1.00930623]
[200.98234329]
[247.65806452]
[1.00930623]
[226.95824386]
[288.99001041]
[1.00930623]
[251.53143617]
[330.3219563]
[1.00930623]
[274.96574426]
[371.65390219]
[1.00930623]
[297.44875843]
[412.98584807]
[1.00930623]
[319.1198762]
[454.31779396]
[1.00930623]
[340.08625285]
[495.64973985]
[1.00930623]
[360.43250283]
[536.98168574]
[1.00930623]
[380.22691786]
[578.31363163]
[1.00930623]
[399.52562417]
[619.64557752]
[1.00930623]
[418.37546028]
[660.97752341]
[1.00930623]
[436.8160281]
[702.3094693]
[1.00930623]
[454.88119151]
[743.64141519]
[1.00930623]
[472.600195]
[784.97336108]
[1.00930623]
[489.99851437]
[826.30530697]
[1.00930623]
[507.09851439]
[867.63725286]
[1.00930623]
[523.91996465]
[908.96919875]
[1.00930623]
[540.48044948]
[950.30114464]
[1.00930623]
[556.79569745]
[991.63309053]
[1.00930623]
[572.87984912]
[1032.96503642]
[1.00930623]
[588.7456766]
[1074.29698231]
[1

In [340]:
%timeit model.evaluate_blocks(py=True)

[3471.54984391]
[1.00930623]
[1326.87102351]
[3512.8817898]
[1.00930623]
[1337.46155663]
[3554.21373569]
[1.00930623]
[1348.01166371]
[3595.54568158]
[1.00930623]
[1358.5219696]
[3636.87762747]
[1.00930623]
[1368.99308243]
[3678.20957336]
[1.00930623]
[1379.42559426]
[3719.54151925]
[1.00930623]
[1389.82008162]
[3760.87346514]
[1.00930623]
[1400.17710614]
1.32 s ± 26.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
